In [1]:
import pandas as pd
import numpy as np
from scipy.interpolate import CubicSpline, UnivariateSpline, InterpolatedUnivariateSpline, interp1d
from datetime import datetime
from math import floor
import matplotlib.pyplot as plt
from BEE_WEATHER_DATA import BROODMINDER_GET, AMBIENT_GET, READ_HIVE, PROCESS_HIVE, READ_BEE_WEATHER, PROCESS_BEE_WEATHER, PROCESS_AMBIENT, GRAPH_DATA, GET_MOON_IMAGE, GET_FORECAST, PROCESS_FORECAST

hive_name = "New Left Hive"
key = 1

if key == 1:
    Data = PROCESS_AMBIENT()
    display(Data)
    #GRAPH_DATA(Data)
if key == 2:
    Data = PROCESS_BEE_WEATHER()
    #display(Data)
    GRAPH_DATA(Data)
if key == 3:
    Data = PROCESS_HIVE(hive_name, interp=1)
    #display(Data)
    [GRAPH_DATA(Data[str(i)]) for i in Data.keys()]
if key == 4:
    pix = GET_MOON_IMAGE(216, save=1)
    plt.imshow(pix)
    plt.show()
if key == 5:
    data = GET_FORECAST()
    response = PROCESS_FORECAST(data)
    display(response)

print("Done")

,dateutc,winddir,windspeedmph,windgustmph,maxdailygust,tempf,humidity,hourlyrainin,eventrainin,dailyrainin,weeklyrainin,monthlyrainin,yearlyrainin,totalrainin,uv,solarradiation,feelsLike,dewPoint,lastRain
8063,1.654214e+09,298,0.00,1.12,13.65,75.2,73,0.0,0.00,0.00,0.19,0.00,2.93,2.93,0,41.59,75.85,65.94,2022-05-31T00:29:00.000Z
8064,1.654215e+09,350,0.67,1.12,13.65,75.0,73,0.0,0.00,0.00,0.19,0.00,2.93,2.93,0,35.48,75.63,65.74,2022-05-31T00:29:00.000Z
8065,1.654215e+09,347,0.00,0.00,13.65,74.8,74,0.0,0.00,0.00,0.19,0.00,2.93,2.93,0,30.34,75.46,65.94,2022-05-31T00:29:00.000Z
8066,1.654215e+09,3,0.45,1.12,13.65,74.7,74,0.0,0.00,0.00,0.19,0.00,2.93,2.93,0,22.51,75.35,65.85,2022-05-31T00:29:00.000Z
8067,1.654216e+09,224,1.12,4.47,13.65,74.5,75,0.0,0.00,0.00,0.19,0.00,2.93,2.93,0,18.11,75.18,66.04,2022-05-31T00:29:00.000Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10074,1.654818e+09,338,0.22,2.24,11.41,79.5,60,0.0,0.09,0.04,0.61,0.61,3.54,3.54,0,44.65,79.97,64.38,2022-06-09T08:41:00.000Z
10075,1.654818e+09,1,0.67,2.24,11.41,79.3,60,0.0,0.09,0.04,0.61,0.61,3.54,3.54,0,41.10,79.75,64.19,2022-06-09T08:41:00.000Z
10076,1.654819e+09,343,0.22,5.82,11.41,79.0,59,0.0,0.09,0.04,0.61,0.61,3.54,3.54,0,37.68,79.37,63.43,2022-06-09T08:41:00.000Z
10077,1.654819e+09,15,1.79,6.93,11.41,78.8,61,0.0,0.09,0.04,0.61,0.61,3.54,3.54,0,32.42,79.25,64.19,2022-06-09T08:41:00.000Z


Done


In [2]:
import ssl
import requests
import time as t
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import os
import pandas as pd
from glob import glob
import numpy as np
from scipy.interpolate import CubicSpline, UnivariateSpline, InterpolatedUnivariateSpline, interp1d, splrep, PchipInterpolator
import re
from math import floor
import matplotlib.pyplot as plt
#from decouple import config
import requests
from io import BytesIO
from PIL import Image
import json
from datetime import date, datetime, timedelta, timezone
import urllib.request
import ssl
import re
import array
import time as t
import csv
from csv import writer
import numpy as np
import itertools
from send2trash import send2trash
import dns.resolver
import pytz
import astral, astral.sun

# Get Ambient data via URL and format
print("Getting Ambient data.")
# csv file name
filename = "Ambient/Ambient_Data.csv"

# Define weather station data
AMBIENT_API_KEY = "a6ed012c794e4d8283cb2475e187daaffb847bc9d13d4e13a90d5b4c2e11c98a"
AMBIENT_APPLICATION_KEY = "d34a5337cdbe460a8624135e6661bdb4bd810a078dc84f47b07c0170bc867eec"
AMBIENT_MAC = "00:0E:C6:30:1F:CC"

# Read in current csv values
rows = pd.read_csv(filename, sep=',', on_bad_lines='skip')

AMBIENT_START_DATE = datetime.fromtimestamp(int(rows['dateutc'][len(rows['dateutc'])-1]))

print(AMBIENT_START_DATE)
# Determine number of days since beginning of record
dateDelta = datetime.today() - AMBIENT_START_DATE

# Iterate through each day of data
for i in range(dateDelta.days + 1):
    # Fetch data from Ambient
    AMBIENT_DATE = AMBIENT_START_DATE + timedelta(days=i)
    AMBIENT_DATE = str(AMBIENT_DATE)
    AMBIENT_DATE = AMBIENT_DATE[:AMBIENT_DATE.rindex(" ")]
    print(AMBIENT_DATE)
    query_params = {'applicationKey': AMBIENT_APPLICATION_KEY, 'macAddress': AMBIENT_MAC, 'apiKey': AMBIENT_API_KEY, 'endDate': AMBIENT_DATE}
    AMBIENT_ENDPOINT = f'https://rt.ambientweather.net/v1/devices/{AMBIENT_MAC}?apiKey={AMBIENT_API_KEY}&applicationKey={AMBIENT_APPLICATION_KEY}&endDate={AMBIENT_DATE}'
    data = pd.DataFrame()

    response = requests.get(AMBIENT_ENDPOINT)#, params=query_params)
    while True:
        if response.status_code == 200:
            data = pd.DataFrame(response.json())
            break
        else:
            # Handle errors
            print(f'Error: {response.status_code} - {response.text}')
        
    # Wait for 1 second to comply with Ambient server limits
    t.sleep(1)
    
    # Sort and append new data
    if data.empty:
        pass
    else:
        data = data[['dateutc', 'winddir', 'windspeedmph', 'windgustmph', 'maxdailygust',
                     'tempf', 'humidity', 'hourlyrainin', 'eventrainin', 'dailyrainin',
                     'weeklyrainin', 'monthlyrainin', 'yearlyrainin', 'totalrainin', 'uv',
                     'solarradiation', 'feelsLike', 'dewPoint', 'lastRain']]
        data['dateutc'] = (data['dateutc'].astype(np.int64) / 1000.0)
        sorted_df = data.sort_values('dateutc', axis=0, ascending=True, kind='mergesort')
        sorted_df.to_csv(filename, mode='a', index=False, header=False)


# Re-read file and drop any duplicates
rows = pd.read_csv(filename, sep=',', on_bad_lines='skip')
rows = rows.drop_duplicates(subset=['dateutc'], keep='last').reset_index(drop=True)
rows.to_csv(filename, mode='w', index=False, header=True)

Getting Ambient data.
Index(['dateutc', 'winddir', 'windspeedmph', 'windgustmph', 'maxdailygust',
       'tempf', 'humidity', 'hourlyrainin', 'eventrainin', 'dailyrainin',
       'weeklyrainin', 'monthlyrainin', 'yearlyrainin', 'totalrainin', 'uv',
       'solarradiation', 'feelsLike', 'dewPoint', 'lastRain'],
      dtype='object')
2022-06-09 18:00:00
2022-06-09
2022-06-10
2022-06-11
2022-06-12
2022-06-13
2022-06-14
2022-06-15
2022-06-16
2022-06-17
2022-06-18
2022-06-19
2022-06-20
2022-06-21
2022-06-22
2022-06-23
2022-06-24
2022-06-25
2022-06-26
2022-06-27
2022-06-28
2022-06-29
2022-06-30
2022-07-01
2022-07-02
2022-07-03
2022-07-04
2022-07-05
2022-07-06
2022-07-07
2022-07-08
2022-07-09
2022-07-10
2022-07-11
2022-07-12
2022-07-13
2022-07-14
2022-07-15
2022-07-16
2022-07-17
2022-07-18
2022-07-19
2022-07-20
2022-07-21
2022-07-22
2022-07-23
2022-07-24
2022-07-25
2022-07-26
2022-07-27
2022-07-28
2022-07-29
2022-07-30
2022-07-31
2022-08-01
2022-08-02
2022-08-03
2022-08-04
2022-08-05
2022-08-

In [ ]:
https://rt.ambientweather.net/v1/devices/00:0E:C6:30:1F:CC?apiKey=a6ed012c794e4d8283cb2475e187daaffb847bc9d13d4e13a90d5b4c2e11c98a&applicationKey=d34a5337cdbe460a8624135e6661bdb4bd810a078dc84f47b07c0170bc867eec&endDate=2022-06-09